In [21]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
import time
import json 
import numpy as np

# Extract (Scrape) Data

#### *Does not need to be loaded to pymongo* - can be scraped on the spot for the Project 2

In [ ]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [ ]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

# Load - Transform

### Load dump to mongo / Clean on the go

In [ ]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
# db.nba_players_stg.drop()
# db.nba_news_live.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

### Validate data availability

In [ ]:

# result_dict['facts'] = list(db.mars_fact.find({}, {"_id": 0}))
for a in db.nba_players_stg.find({}, {"_id": 0}).limit(2):
    print(a)

## Scrape News

In [ ]:
nba_news_url = "https://apnews.com/NBA"
browser.visit(nba_news_url)
browser.html
# time.sleep(10)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_tables = soup.find_all('div', attrs={"data-key":"feed-card-wire-story-with-image"})
# schedules_sites = pd.read_html(sched_tables.prettify())
# schedules_sites


In [ ]:
for row in news_tables:
    news_content = {}
    news_content['headline'] = row.h1.text
    news_content['content'] = row.p.text
    print(f"Loading {row.h1.text}....")
    db.nba_news_live.insert_one(news_content)

### ETL For elections

In [ ]:
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_pd = pd.read_json(geojson_source)

# json_file = "etl/gz_2010_us_050_00_500k.json"
# customer_location_df = pd.read_json(json_file, encoding="utf-8")

In [ ]:
results_file = "etl/countypres_2000-2016.csv"
results_df = pd.read_csv(results_file, encoding="utf-8")
results_df.head()
results_df.dropna(inplace=True)
results_df.drop(columns=['office', 'version'],inplace=True)
results_df.head()

In [ ]:
# results_df.loc[results_df['county'].str.lower() =="Orange".lower(),:]

# Cast float to integer
results_df.FIPS = results_df.FIPS.astype(int)
results_df.candidatevotes = results_df.candidatevotes.astype(int)
results_df.party = results_df.party.str.title()
# Calculate percent won
results_df["percentwon"] = ((results_df.candidatevotes / results_df.totalvotes) * 100).map("{0:,.2f}".format).astype(float)
# results_df.percentwon = results_df.percentwon.astype(int)
# Drop index
results_df.reset_index(drop=True, inplace=True)

# Test results
# results_df.loc[(results_df['county'].str.lower() =="Orange".lower()) & (results_df['state'] == "California"),:].head(100)
# results_df.head()
# results_2016 = results_df.loc[(results_df.year == 2016) & (results_df['state'] == "California"),:]
#results_2016 = results_df.loc[(results_df.year >= 2008) & (results_df.year <= 2016) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]

In [ ]:
results_2016

In [ ]:
# results_df.to_json("etl/processed.json", orient='records')

## GEO JSON BIG SOURCE

In [ ]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/us-county-boundaries.csv"
geojson_pd = pd.read_csv(geojson_source, delimiter=';')
geojson_pd.dtypes
geojson_pd = geojson_pd[['GEOID','Geo Shape']]

### GEOJSON Small source

In [2]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/gz_2010_us_050_00_20m.json"
geojson_pd = pd.read_json(geojson_source, encoding='latin-1', lines=True)
geojson_pd.rename(columns={"geometry": "Geo Shape", "GEO_ID": "geoid"},inplace=True)
# geojson_pd = geojson_pd[['Geometry']]
# geojson_pd['FIPS'] = int(geojson_pd['properties']['GEO_ID'][-5:])
geojson_pd['FIPS']  = geojson_pd.apply(lambda row: int(row.properties['GEO_ID'][-5:]), axis=1)
geojson_pd['STATEID']  = geojson_pd.apply(lambda row: int(row.properties['STATE']), axis=1)
geojson_pd['geoid']  = geojson_pd.apply(lambda row: row.properties['GEO_ID'], axis=1)
# geojson_pd['properties'][0]['GEO_ID'] '0500000US01001'
# int(geojson_pd['properties'][0]['GEO_ID'][-5:])
# geojson_pd

In [3]:

# geojson_source = "etl/us_states.json"
# geojson_pd = pd.read_json(geojson_source, lines=True)
# geojson_pd
california_geojson_pd = geojson_pd.loc[(geojson_pd.STATEID == 6),:].reset_index()
california_geojson_pd

,index,properties,Geo Shape,FIPS,STATEID,geoid
0,23,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005
1,24,"{'GEO_ID': '0500000US06021', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.937...",6021,6,0500000US06021
2,25,"{'GEO_ID': '0500000US06033', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.627...",6033,6,0500000US06033
3,26,"{'GEO_ID': '0500000US06043', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.387...",6043,6,0500000US06043
4,27,"{'GEO_ID': '0500000US06055', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.103...",6055,6,0500000US06055
5,28,"{'GEO_ID': '0500000US06089', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-123.065...",6089,6,0500000US06089
6,29,"{'GEO_ID': '0500000US06099', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.931...",6099,6,0500000US06099
7,30,"{'GEO_ID': '0500000US06115', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.333...",6115,6,0500000US06115
8,684,"{'GEO_ID': '0500000US06003', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.072...",6003,6,0500000US06003
9,685,"{'GEO_ID': '0500000US06007', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.076...",6007,6,0500000US06007


In [4]:
results_file = "etl/california-results.csv"
results_df = pd.read_csv(results_file, encoding="utf-8")
results_df.head()

,fips,year,winner
0,6001,2004,1
1,6003,2004,1
2,6005,2004,-1
3,6007,2004,-1
4,6009,2004,-1


In [6]:
geojson_pd.head(10)
merged_df = geojson_pd.merge(results_df, how="inner", right_on='fips', left_on='FIPS');
merged_df

,properties,Geo Shape,FIPS,STATEID,geoid,fips,year,winner
0,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005,6005,2004,-1
1,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005,6005,2008,-1
2,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005,6005,2012,-1
3,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005,6005,2016,-1
4,"{'GEO_ID': '0500000US06021', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.937...",6021,6,0500000US06021,6021,2004,-1
...,...,...,...,...,...,...,...,...
227,"{'GEO_ID': '0500000US06017', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.141...",6017,6,0500000US06017,6017,2016,-1
228,"{'GEO_ID': '0500000US06085', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.654...",6085,6,0500000US06085,6085,2004,1
229,"{'GEO_ID': '0500000US06085', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.654...",6085,6,0500000US06085,6085,2008,1
230,"{'GEO_ID': '0500000US06085', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.654...",6085,6,0500000US06085,6085,2012,1


In [7]:
# create 8 new dataframes
merged_df_2004_dem = merged_df.loc[(merged_df['year'] == 2004) & (merged_df['winner'] == 1) ,:]
#merged_df_2004_rep
merged_df_2004_rep = merged_df.loc[(merged_df['year'] == 2004) & (merged_df['winner'] == -1) ,:]
#-2008
merged_df_2008_dem = merged_df.loc[(merged_df['year'] == 2008) & (merged_df['winner'] == 1) ,:]
#merged_df_2008_rep
merged_df_2008_rep = merged_df.loc[(merged_df['year'] == 2008) & (merged_df['winner'] == -1) ,:]
#2012
merged_df_2012_dem = merged_df.loc[(merged_df['year'] == 2012) & (merged_df['winner'] == 1) ,:]
#merged_df_2012_rep
merged_df_2012_rep = merged_df.loc[(merged_df['year'] == 2012) & (merged_df['winner'] == -1) ,:]
#2016
merged_df_2016_dem = merged_df.loc[(merged_df['year'] == 2016) & (merged_df['winner'] == 1) ,:]
#merged_df_2016_rep
merged_df_2016_rep = merged_df.loc[(merged_df['year'] == 2016) & (merged_df['winner'] == -1) ,:]

In [8]:
merged_df_2016_rep

,properties,Geo Shape,FIPS,STATEID,geoid,fips,year,winner
3,"{'GEO_ID': '0500000US06005', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.995...",6005,6,0500000US06005,6005,2016,-1
7,"{'GEO_ID': '0500000US06021', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.937...",6021,6,0500000US06021,6021,2016,-1
15,"{'GEO_ID': '0500000US06043', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.387...",6043,6,0500000US06043,6043,2016,-1
23,"{'GEO_ID': '0500000US06089', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-123.065...",6089,6,0500000US06089,6089,2016,-1
31,"{'GEO_ID': '0500000US06115', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.333...",6115,6,0500000US06115,6115,2016,-1
39,"{'GEO_ID': '0500000US06007', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.076...",6007,6,0500000US06007,6007,2016,-1
47,"{'GEO_ID': '0500000US06049', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.447...",6049,6,0500000US06049,6049,2016,-1
99,"{'GEO_ID': '0500000US06101', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-121.484...",6101,6,0500000US06101,6101,2016,-1
103,"{'GEO_ID': '0500000US06103', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-122.934...",6103,6,0500000US06103,6103,2016,-1
107,"{'GEO_ID': '0500000US06109', 'STATE': '06', 'C...","{'type': 'Polygon', 'coordinates': [[[-120.652...",6109,6,0500000US06109,6109,2016,-1


In [9]:
# geojson_pd
# results_df
def geo_merger(results_2016):
    joined_df = pd.DataFrame();
    joined_df = geojson_pd.merge(results_2016[['FIPS','year','state_po','county','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_2016.party == 'Republican')], how="inner", right_on='FIPS', left_on='GEOID');
    joined_df = joined_df.merge(results_2016[['FIPS','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_2016.party == 'Democrat')], how="inner", right_on='FIPS', left_on='GEOID',suffixes=('_rep', '_dem'));
    joined_df['winner'] = np.where(joined_df['candidatevotes_rep'] > joined_df['candidatevotes_dem'], 'red',  'blue');
    return joined_df;


In [ ]:
# results_2008 = results_df.loc[(results_df.year == 2008) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
# results_2012 = results_df.loc[(results_df.year == 2012) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
# results_2016 = results_df.loc[(results_df.year == 2016) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
# merged_2008 = geo_merger(results_2008);
# merged_2012 = geo_merger(results_2012);
# merged_2016 = geo_merger(results_2016);


In [ ]:
# final_merged = pd.concat([merged_2008, merged_2012, merged_2016])

In [ ]:
# final_merged

In [10]:
def df_to_geojson(df, properties):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{}
#                    'geometry':{}
                  }
        feature['geometry'] = row['Geo Shape'] #Use json.loads(row['Geo Shape']) if using the big source
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [13]:
# properties = ['year', 'state_po', 'county', 'candidate_rep', 'party_rep', 'candidatevotes_rep', 'percentwon_rep', 'candidate_dem', 'party_dem', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
# properties = ['year' ,'state_po', 'county', 'candidate_rep', 'party_rep', 'candidatevotes_rep', 'percentwon_rep', 'candidate_dem', 'party_dem', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
properties = ['geoid', 'year' ,'winner']

geojson1a = df_to_geojson(merged_df_2004_dem, properties)
geojson1b = df_to_geojson(merged_df_2004_rep, properties)
geojson2a = df_to_geojson(merged_df_2008_dem, properties)
geojson2b = df_to_geojson(merged_df_2008_rep, properties)
geojson3a = df_to_geojson(merged_df_2012_dem, properties)
geojson3b = df_to_geojson(merged_df_2012_rep, properties)
geojson4a = df_to_geojson(merged_df_2016_dem, properties)
geojson4b = df_to_geojson(merged_df_2016_rep, properties)

In [ ]:
geojson1b

In [16]:
json = json.dumps(geojson1b)
f = open("etl/geojson1b.json","w")
f.write(json)
f.close()

# output_filename = 'dataset.js'
# with open(output_filename, 'wb') as output_file:
#     output_file.write('var dataset = ')
#     json.dump(geojson, output_file, indent=2) 

AttributeError: 'str' object has no attribute 'dumps'

In [28]:
import json 
json = json.dumps(geojson3b)
f = open("etl/geojson3b.json","w")
f.write(json)
f.close()

In [ ]:
geojson_pd